In [4]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
tables = pd.read_html('https://www.prisonpolicy.org/blog/2020/12/02/jail-and-prison-covid-populations/')
print("{} Tables Found.".format(len(tables)))

2 Tables Found.


## Table 1

In [3]:
tables[0].head()

,County,State,March population,July population,Most recent population,Percent change from March to July,Percent change from July to the most recent date,Net percent change since March,March date,July date,Most recent date
0,Autauga,Ala.,171,158,193,-8%,20%,13%,3/10,7/1,11/15
1,Blount,Ala.,125,117,159,-6%,36%,27%,3/10,7/1,11/15
2,Chambers,Ala.,134,70,2,-48%,-97%,-99%,3/10,7/1,11/15
3,Cherokee,Ala.,110,73,76,-34%,4%,-31%,3/10,7/1,11/15
4,Clay,Ala.,38,31,31,-18%,0%,-18%,3/10,7/1,11/15


## Table 2

In [4]:
tables[1].head()

,State,January,January.1,May,May.1,July,July.1,August,August.1,September,September.1,October,October.1,November,November.1,Most recent,Most recent.1,Population Data Source
0,NaN,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,Prison population,Date,NaN
1,Arizona,42441,1/1,41386,5/1,40102,7/1,39125,8/21,38865,9/27,38741,10/7,38562,11/1,38385,11/19,"Vera’s People in Prison, 2019; ADCRR COVID-19 ..."
2,California,125365,1/15,119183,5/6,115201,7/1,104544,8/19,100747,9/30,101003,10/7,101658,11/4,100153,11/18,CDCR 2020 Weekly Total Population Reports
3,Connecticut,12284,1/1,10973,5/1,9945,7/1,9558,8/24,9391,9/30,9344,10/8,9350,11/1,9299,11/19,Department of Correction’s Total Population Co...
4,Georgia,53924,1/3,51294,5/1,49959,7/3,48274,8/21,46814,9/25,47368,10/2,46649,10/30,45893,11/13,GDC Friday Report


## Working From NYU's Raw Data

In [5]:
# response = requests.get('https://raw.githubusercontent.com/publicsafetylab/public-psl-jdi-pops/master/data.csv')
df = pd.read_csv('https://raw.githubusercontent.com/publicsafetylab/public-psl-jdi-pops/master/data.csv')
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier
0,Autauga,182,2020-01-01,AL,AL-Autauga
1,Autauga,184,2020-01-02,AL,AL-Autauga
2,Autauga,178,2020-01-03,AL,AL-Autauga
3,Autauga,180,2020-01-04,AL,AL-Autauga
4,Autauga,184,2020-01-05,AL,AL-Autauga


In [6]:
orig_len = len(df)

## Subset to After March 10th

In [7]:
df['Scrape_Date'] = pd.to_datetime(df['Scrape_Date'])
df = df[df['Scrape_Date'] >= pd.to_datetime("03-10-2020")]
after_mar10 = len(df)
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier
59,Autauga,171,2020-03-10,AL,AL-Autauga
60,Autauga,172,2020-03-11,AL,AL-Autauga
61,Autauga,173,2020-03-12,AL,AL-Autauga
62,Autauga,179,2020-03-13,AL,AL-Autauga
63,Autauga,180,2020-03-14,AL,AL-Autauga


In [8]:
print("{} post march 10th.".format(after_mar10/orig_len))

0.8947359643783301 post march 10th.


## Subset to Jails with >75% days populated with data

In [9]:
ndata_per_jail = df.groupby('Facility_Identifier')['Population'].count()
max_ex = max(ndata_per_jail)
threshold = .75*max_ex
jails_to_include = ndata_per_jail[ndata_per_jail>=threshold].index

df = df[df['Facility_Identifier'].isin(jails_to_include)]
over_75percent_data = len(df)
print(len(df))
df.head()

153255


,County,Population,Scrape_Date,State,Facility_Identifier
59,Autauga,171,2020-03-10,AL,AL-Autauga
60,Autauga,172,2020-03-11,AL,AL-Autauga
61,Autauga,173,2020-03-12,AL,AL-Autauga
62,Autauga,179,2020-03-13,AL,AL-Autauga
63,Autauga,180,2020-03-14,AL,AL-Autauga


### Quantifying Data Drop-Off

In [10]:
percent = str(round(len(jails_to_include)/len(ndata_per_jail),2)*100)
message = "{} of {} jails ({}%) have over 75% data populated post march 10th."
print(message.format(len(jails_to_include),len(ndata_per_jail), percent))

598 of 1016 jails (59.0%) have over 75% data populated post march 10th.


In [11]:
df['Facility_Identifier'].nunique()

598

## Jails with the last date needs to be within 3 months

In [12]:
import datetime
today = pd.to_datetime(datetime.datetime.now().strftime('%m-%d-%Y')) 
three_months_ago = today - datetime.timedelta(days = 90)
three_months_ago

Timestamp('2020-09-08 00:00:00')

In [13]:
jails_most_recent = df.groupby('Facility_Identifier')['Scrape_Date'].max()
jails_to_include = jails_most_recent[jails_most_recent>=three_months_ago].index
df = df[df['Facility_Identifier'].isin(jails_to_include)] #Subset master dataset
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier
59,Autauga,171,2020-03-10,AL,AL-Autauga
60,Autauga,172,2020-03-11,AL,AL-Autauga
61,Autauga,173,2020-03-12,AL,AL-Autauga
62,Autauga,179,2020-03-13,AL,AL-Autauga
63,Autauga,180,2020-03-14,AL,AL-Autauga


## Quantifying Drop Off Again: Generally have seen that all jails that have 75% of data populated also include data from within 3 months:

In [14]:
within_3_months = len(df)
within_3_months / over_75percent_data

1.0

In [38]:
ls

COVID_Prison_Transformations.ipynb
PrisonWeeklyPopulations_as_of_12-03-2020.csv
README.md


## Summarizing Daily Jail Populations

In [15]:
jail_totals = df.groupby('Scrape_Date')['Population'].agg(['count', 'sum'])
today = datetime.datetime.now().strftime('%m-%d-%Y') #Create string of today's date
filename = "Total_Jail_Population_By_Date_as_of_{}.csv".format(today)
jail_totals.to_csv(filename)

In [80]:
jail_totals.columns = ['Count_Of_Jails','Total_Jail_Population']
jail_totals['Seven_Day_Rolling_Average'] = jail_totals['Total_Jail_Population'].rolling(7).mean()
jail_totals.head(20)

,Count_Of_Jails,Total_Jail_Population,Seven_Day_Rolling_Average
Scrape_Date,,,
2020-03-10,447,122586,NaN
2020-03-11,451,122876,NaN
2020-03-12,448,121841,NaN
2020-03-13,442,120915,NaN
2020-03-14,426,118396,NaN
2020-03-15,466,125620,NaN
2020-03-16,471,128115,122907.000000
2020-03-17,470,123752,123073.571429
2020-03-18,472,124433,123296.000000


In [42]:
ls

COVID_Prison_Transformations.ipynb
PrisonWeeklyPopulations_as_of_12-03-2020.csv
README.md
Total_Jail_Population_By_Date_as_of_12-03-2020
Total_Jail_Population_By_Date_as_of_12-03-2020.csv


In [43]:
pwd

'/Users/matthewmitchell/Documents/PPI_COVID_Data_Rollups'

In [44]:
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier,RollingAvg
59,Autauga,171,2020-03-10,AL,AL-Autauga,171.000000
60,Autauga,172,2020-03-11,AL,AL-Autauga,170.857143
61,Autauga,173,2020-03-12,AL,AL-Autauga,170.571429
62,Autauga,179,2020-03-13,AL,AL-Autauga,170.857143
63,Autauga,180,2020-03-14,AL,AL-Autauga,172.857143


In [48]:
def is_weekday(n):
    n = int(n)
    if n == 0 or n == 6:
        return False
    else:
        return True
df['Weekday'] = df['Scrape_Date'].dt.strftime('%w').map(is_weekday)
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier,RollingAvg,Weekday
59,Autauga,171,2020-03-10,AL,AL-Autauga,171.000000,True
60,Autauga,172,2020-03-11,AL,AL-Autauga,170.857143,True
61,Autauga,173,2020-03-12,AL,AL-Autauga,170.571429,True
62,Autauga,179,2020-03-13,AL,AL-Autauga,170.857143,True
63,Autauga,180,2020-03-14,AL,AL-Autauga,172.857143,False


In [ ]:
df[df["Facility_Identifier"]=="WI-Sawyer"].head(50)

## TBD: programatically create list of dates based on most recent weekday (or potentially specific day of week) for each month

## First Day of Each Month

In [64]:
first_days = df.groupby(['Facility_Identifier', 'Month', 'Year']).head(1)
print(len(first_days))
first_days.head()


5827


,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
59,Autauga,171,2020-03-10,AL,AL-Autauga,03,2020,Tuesday
81,Autauga,150,2020-04-01,AL,AL-Autauga,04,2020,Wednesday
111,Autauga,155,2020-05-01,AL,AL-Autauga,05,2020,Friday
142,Autauga,173,2020-06-01,AL,AL-Autauga,06,2020,Monday
172,Autauga,158,2020-07-01,AL,AL-Autauga,07,2020,Wednesday


In [69]:
first_days['Scrape_Date'] = "FirstDay" + first_days['Month'] + "-" + first_days['Year']
first_days.head()

/Users/matthewmitchell/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
59,Autauga,171,FirstDay03-2020,AL,AL-Autauga,03,2020,Tuesday
81,Autauga,150,FirstDay04-2020,AL,AL-Autauga,04,2020,Wednesday
111,Autauga,155,FirstDay05-2020,AL,AL-Autauga,05,2020,Friday
142,Autauga,173,FirstDay06-2020,AL,AL-Autauga,06,2020,Monday
172,Autauga,158,FirstDay07-2020,AL,AL-Autauga,07,2020,Wednesday


## First Monday of Each Month

In [19]:
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier
59,Autauga,171,2020-03-10,AL,AL-Autauga
60,Autauga,172,2020-03-11,AL,AL-Autauga
61,Autauga,173,2020-03-12,AL,AL-Autauga
62,Autauga,179,2020-03-13,AL,AL-Autauga
63,Autauga,180,2020-03-14,AL,AL-Autauga


In [57]:
df['Month'] = df.Scrape_Date.dt.strftime('%m')
df['Year'] = df.Scrape_Date.dt.strftime('%Y')
df['DayOfWeek'] = df.Scrape_Date.dt.strftime('%A')
mondays = df[df.DayOfWeek=='Monday']
first_mondays = mondays.groupby(['Facility_Identifier', 'Month', 'Year']).head(1)
first_mondays.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
65,Autauga,178,2020-03-16,AL,AL-Autauga,03,2020,Monday
86,Autauga,138,2020-04-06,AL,AL-Autauga,04,2020,Monday
114,Autauga,156,2020-05-04,AL,AL-Autauga,05,2020,Monday
142,Autauga,173,2020-06-01,AL,AL-Autauga,06,2020,Monday
177,Autauga,169,2020-07-06,AL,AL-Autauga,07,2020,Monday


In [58]:
most_recent = df.groupby(['Facility_Identifier']).tail(1)
most_recent.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
324,Autauga,181,2020-12-06,AL,AL-Autauga,12,2020,Sunday
652,Blount,155,2020-12-06,AL,AL-Blount,12,2020,Sunday
859,Calhoun,579,2020-12-06,AL,AL-Calhoun,12,2020,Sunday
1128,Chambers,2,2020-12-06,AL,AL-Chambers,12,2020,Sunday
1455,Cherokee,89,2020-12-06,AL,AL-Cherokee,12,2020,Sunday


In [27]:
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
59,Autauga,171,2020-03-10,AL,AL-Autauga,March,2020,Tuesday
60,Autauga,172,2020-03-11,AL,AL-Autauga,March,2020,Wednesday
61,Autauga,173,2020-03-12,AL,AL-Autauga,March,2020,Thursday
62,Autauga,179,2020-03-13,AL,AL-Autauga,March,2020,Friday
63,Autauga,180,2020-03-14,AL,AL-Autauga,March,2020,Saturday


In [29]:
df.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
59,Autauga,171,2020-03-10,AL,AL-Autauga,March,2020,Tuesday
60,Autauga,172,2020-03-11,AL,AL-Autauga,March,2020,Wednesday
61,Autauga,173,2020-03-12,AL,AL-Autauga,March,2020,Thursday
62,Autauga,179,2020-03-13,AL,AL-Autauga,March,2020,Friday
63,Autauga,180,2020-03-14,AL,AL-Autauga,March,2020,Saturday


## Verify DataFrame Correctly Sorted by Facility/Date

In [35]:
all(df == df.sort_values(by=['Facility_Identifier', 'Scrape_Date']))

True

## Ensure there is one per Facility

In [34]:
len(most_recent) == df.Facility_Identifier.nunique()

True

## March 10th

In [59]:
march_10 = df[df.Scrape_Date == pd.to_datetime("03-10-2020")]
march_10.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
59,Autauga,171,2020-03-10,AL,AL-Autauga,03,2020,Tuesday
385,Blount,125,2020-03-10,AL,AL-Blount,03,2020,Tuesday
865,Chambers,134,2020-03-10,AL,AL-Chambers,03,2020,Tuesday
1192,Cherokee,110,2020-03-10,AL,AL-Cherokee,03,2020,Tuesday
1836,Clay,38,2020-03-10,AL,AL-Clay,03,2020,Tuesday


## First Mondays Summary

In [68]:
first_monday_summary = first_mondays.groupby(['Facility_Identifier', 'Month', 'Year']).head(1)
first_monday_summary['Scrape_Date'] = "FirstMonday" + first_monday_summary['Month'] + "-" + first_monday_summary['Year']
first_monday_summary.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
65,Autauga,178,FirstMonday03-2020,AL,AL-Autauga,03,2020,Monday
86,Autauga,138,FirstMonday04-2020,AL,AL-Autauga,04,2020,Monday
114,Autauga,156,FirstMonday05-2020,AL,AL-Autauga,05,2020,Monday
142,Autauga,173,FirstMonday06-2020,AL,AL-Autauga,06,2020,Monday
177,Autauga,169,FirstMonday07-2020,AL,AL-Autauga,07,2020,Monday


## Stitching Together the Final Summary



In [70]:
pre_pivot = pd.concat([march_10, first_mondays, most_recent, first_monday_summary, first_days], ignore_index=True)
pre_pivot.head()

,County,Population,Scrape_Date,State,Facility_Identifier,Month,Year,DayOfWeek
0,Autauga,171,2020-03-10 00:00:00,AL,AL-Autauga,03,2020,Tuesday
1,Blount,125,2020-03-10 00:00:00,AL,AL-Blount,03,2020,Tuesday
2,Chambers,134,2020-03-10 00:00:00,AL,AL-Chambers,03,2020,Tuesday
3,Cherokee,110,2020-03-10 00:00:00,AL,AL-Cherokee,03,2020,Tuesday
4,Clay,38,2020-03-10 00:00:00,AL,AL-Clay,03,2020,Tuesday


In [71]:
final = pre_pivot.pivot(index="Facility_Identifier", columns="Scrape_Date", values="Population")
final.head()

Scrape_Date,2020-03-10 00:00:00,2020-03-16 00:00:00,2020-03-23 00:00:00,2020-03-30 00:00:00,2020-04-06 00:00:00,2020-04-13 00:00:00,2020-04-20 00:00:00,2020-04-27 00:00:00,2020-05-04 00:00:00,2020-05-11 00:00:00,...,FirstDay12-2020,FirstMonday03-2020,FirstMonday04-2020,FirstMonday05-2020,FirstMonday06-2020,FirstMonday07-2020,FirstMonday08-2020,FirstMonday09-2020,FirstMonday10-2020,FirstMonday11-2020
Facility_Identifier,,,,,,,,,,,,,,,,,,,,,
AL-Autauga,171.0,178.0,NaN,NaN,138.0,NaN,NaN,NaN,156.0,NaN,...,190.0,178.0,138.0,156.0,173.0,169.0,146.0,160.0,159.0,184.0
AL-Blount,125.0,136.0,NaN,NaN,90.0,NaN,NaN,NaN,81.0,NaN,...,157.0,136.0,90.0,81.0,107.0,117.0,119.0,148.0,147.0,157.0
AL-Calhoun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,419.0,...,584.0,NaN,NaN,419.0,436.0,444.0,511.0,571.0,559.0,567.0
AL-Chambers,134.0,143.0,NaN,NaN,108.0,NaN,NaN,NaN,102.0,NaN,...,2.0,143.0,108.0,102.0,87.0,68.0,51.0,2.0,2.0,2.0
AL-Cherokee,110.0,123.0,NaN,NaN,71.0,NaN,NaN,NaN,67.0,NaN,...,83.0,123.0,71.0,67.0,71.0,66.0,80.0,79.0,74.0,79.0


## Alternative Formulations:

|Facility | Month | Year | Average_Population | Most_Recent_Date | Most_Recent_Population | N_Days_Data|
| :----- | :---: |:---: | :-----------------:| :---------------: | :--------------------:| -----------:|

Other potential columns:

* First Monday
* First Weekday
* First Day of Month
* High Population
* Lowest Population


Other potential interesting stats:

* reported population differences weekday vs weekend
* percent of facilities reporting data on weekends / Holidays vs weekdays

## Summary By Facility

In [75]:
pd.DataFrame(df.groupby('Facility_Identifier')['Population'].mean())

,Population
Facility_Identifier,
AL-Autauga,162.992481
AL-Blount,121.593284
AL-Calhoun,514.371981
AL-Chambers,50.965909
AL-Cherokee,77.750000
...,...
WI-Polk,65.422727
WI-Racine,597.435115
WI-Sawyer,79.943548


## Weekly Rolling Averages

In [4]:
grouped = df.groupby(['Facility_Identifier'])
for i, (name, g) in enumerate(grouped):
        temp = g['Population'].rolling(7).mean()
        for idx in temp.index:
            df.at[idx, 'RollingAvg'] = temp.loc[idx]
df.head(10)

,County,Population,Scrape_Date,State,Facility_Identifier,RollingAvg
0,Autauga,182,2020-01-01,AL,AL-Autauga,NaN
1,Autauga,184,2020-01-02,AL,AL-Autauga,NaN
2,Autauga,178,2020-01-03,AL,AL-Autauga,NaN
3,Autauga,180,2020-01-04,AL,AL-Autauga,NaN
4,Autauga,184,2020-01-05,AL,AL-Autauga,NaN
5,Autauga,183,2020-01-06,AL,AL-Autauga,NaN
6,Autauga,180,2020-01-07,AL,AL-Autauga,181.571429
7,Autauga,180,2020-01-08,AL,AL-Autauga,181.285714
8,Autauga,179,2020-01-09,AL,AL-Autauga,180.571429
9,Autauga,183,2020-01-10,AL,AL-Autauga,181.285714


## Save Transformed Output to Excel

In [73]:
import datetime
today = datetime.datetime.now().strftime('%m-%d-%Y') #Create string of today's date
print(today)

12-07-2020


In [74]:
final.head()

Scrape_Date,2020-03-10 00:00:00,2020-03-16 00:00:00,2020-03-23 00:00:00,2020-03-30 00:00:00,2020-04-06 00:00:00,2020-04-13 00:00:00,2020-04-20 00:00:00,2020-04-27 00:00:00,2020-05-04 00:00:00,2020-05-11 00:00:00,...,FirstDay12-2020,FirstMonday03-2020,FirstMonday04-2020,FirstMonday05-2020,FirstMonday06-2020,FirstMonday07-2020,FirstMonday08-2020,FirstMonday09-2020,FirstMonday10-2020,FirstMonday11-2020
Facility_Identifier,,,,,,,,,,,,,,,,,,,,,
AL-Autauga,171.0,178.0,NaN,NaN,138.0,NaN,NaN,NaN,156.0,NaN,...,190.0,178.0,138.0,156.0,173.0,169.0,146.0,160.0,159.0,184.0
AL-Blount,125.0,136.0,NaN,NaN,90.0,NaN,NaN,NaN,81.0,NaN,...,157.0,136.0,90.0,81.0,107.0,117.0,119.0,148.0,147.0,157.0
AL-Calhoun,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,419.0,...,584.0,NaN,NaN,419.0,436.0,444.0,511.0,571.0,559.0,567.0
AL-Chambers,134.0,143.0,NaN,NaN,108.0,NaN,NaN,NaN,102.0,NaN,...,2.0,143.0,108.0,102.0,87.0,68.0,51.0,2.0,2.0,2.0
AL-Cherokee,110.0,123.0,NaN,NaN,71.0,NaN,NaN,NaN,67.0,NaN,...,83.0,123.0,71.0,67.0,71.0,66.0,80.0,79.0,74.0,79.0


In [81]:
ls

COVID_Prison_Transformations.ipynb
PrisonWeeklyPopulations_as_of_12-03-2020.csv
README.md
Total_Jail_Population_By_Date_as_of_12-03-2020.csv
Total_Jail_Population_By_Date_as_of_12-07-2020.csv


In [82]:
filename = "Jail_Summaries_as_of_{}.xlsx".format(today)
print('Saving to: {}'.format(filename))

with pd.ExcelWriter(filename) as writer1:
    final.to_excel(writer1, sheet_name = 'FirstMondays')
    jail_totals.to_excel(writer1, sheet_name = 'DailyJailTotals')

Saving to: Jail_Summaries_as_of_12-07-2020.xlsx


In [83]:
ls

COVID_Prison_Transformations.ipynb
Jail_Summaries_as_of_12-07-2020.xlsx
PrisonWeeklyPopulations_as_of_12-03-2020.csv
README.md
Total_Jail_Population_By_Date_as_of_12-03-2020.csv
Total_Jail_Population_By_Date_as_of_12-07-2020.csv


In [14]:
df.to_csv('PrisonWeeklyPopulations_as_of_{}.csv'.format(today)) #Save to file